In [60]:
import requests
import pandas as pd
import numpy as np
import json

In [61]:
counter = 0

In [62]:
#Query all data from a year
#Add counter for the number of extractions
data=[]
for i in range(1,30):
    counter+=1
    url = "https://api-football-v1.p.rapidapi.com/v3/players"

    querystring = {"league":"39","season":"2016","page":str(i)}

    headers = {
        "X-RapidAPI-Host": "api-football-v1.p.rapidapi.com",
        "X-RapidAPI-Key": "d356c39fe7mshf3ec31c0312f244p17df85jsn2bc9fd933f3f"
   }

    response = requests.request("GET", url, headers=headers, params=querystring)
    
    if response.text == "'{'message': 'You have exceeded the rate limit per minute for your plan, BASIC, by the API provider'}'":
        print('Restart next loop at:', i)
        break
    else: 
        data.append(response.text)
else: 
    print('Start next loop at 30')
data
#Good but have to pop last pages which were over my quota per minute

Start next loop at 30


['{"get":"players","parameters":{"league":"39","page":"1","season":"2016"},"errors":[],"results":20,"paging":{"current":1,"total":34},"response":[{"player":{"id":18774,"name":"W. Morgan","firstname":"Wes","lastname":"Morgan","age":37,"birth":{"date":"1984-01-21","place":"Nottingham","country":"England"},"nationality":"Jamaica","height":"186 cm","weight":"101 kg","injured":false,"photo":"https:\\/\\/media.api-sports.io\\/football\\/players\\/18774.png"},"statistics":[{"team":{"id":46,"name":"Leicester","logo":"https:\\/\\/media.api-sports.io\\/football\\/teams\\/46.png"},"league":{"id":39,"name":"Premier League","country":"England","logo":"https:\\/\\/media.api-sports.io\\/football\\/leagues\\/39.png","flag":"https:\\/\\/media.api-sports.io\\/flags\\/gb.svg","season":2016},"games":{"appearences":27,"lineups":27,"minutes":2430,"number":null,"position":"Defender","rating":"6.811111","captain":false},"substitutes":{"in":0,"out":0,"bench":0},"shots":{"total":14,"on":3},"goals":{"total":1,"c

In [63]:
counter

29

In [64]:
#complete year array
for i in range(30,35):
    counter+=1
    url = "https://api-football-v1.p.rapidapi.com/v3/players"
    
    #YEAR TO CHANGE IN QUERY
    querystring = {"league":"39","season":"2016","page":str(i)}

    headers = {
        "X-RapidAPI-Host": "api-football-v1.p.rapidapi.com",
        "X-RapidAPI-Key": "d356c39fe7mshf3ec31c0312f244p17df85jsn2bc9fd933f3f"
    }

    response = requests.request("GET", url, headers=headers, params=querystring)
    
    data.append(response.text)
    
data

['{"get":"players","parameters":{"league":"39","page":"1","season":"2016"},"errors":[],"results":20,"paging":{"current":1,"total":34},"response":[{"player":{"id":18774,"name":"W. Morgan","firstname":"Wes","lastname":"Morgan","age":37,"birth":{"date":"1984-01-21","place":"Nottingham","country":"England"},"nationality":"Jamaica","height":"186 cm","weight":"101 kg","injured":false,"photo":"https:\\/\\/media.api-sports.io\\/football\\/players\\/18774.png"},"statistics":[{"team":{"id":46,"name":"Leicester","logo":"https:\\/\\/media.api-sports.io\\/football\\/teams\\/46.png"},"league":{"id":39,"name":"Premier League","country":"England","logo":"https:\\/\\/media.api-sports.io\\/football\\/leagues\\/39.png","flag":"https:\\/\\/media.api-sports.io\\/flags\\/gb.svg","season":2016},"games":{"appearences":27,"lineups":27,"minutes":2430,"number":null,"position":"Defender","rating":"6.811111","captain":false},"substitutes":{"in":0,"out":0,"bench":0},"shots":{"total":14,"on":3},"goals":{"total":1,"c

In [65]:
from json import loads
#Data2020 is complete, next step is to decide how to model and keep which part of the data
#data2020Json = list(map(lambda i: data2020[i].json(),data2020))
tableau=[]
for i in range(len(data)):
    tableau.append(i)
#Format data to json
jsonData=list(map(lambda i: loads(data[i]),tableau))
len(jsonData)

34

In [67]:
len(jsonData[33]['response'])

18

In [68]:
#Create prototype array which will be column names in the df
proto= ["id"\
, "firstname"\
, "lastname"\
, "age"\
, "nationality"\
, "height"\
, "weight"\
, "injured"\
, "teamID"\
, "teamname"\
, "appearences"\
, "lineups"\
, "minutes"\
, "position"\
, "rating"\
, "captain"\
, "subIn"\
, "subOut"\
, "bench"\
, "totalShots"\
, "shotsOn"\
, "totalGoals"\
, "goalsConceded"\
, "goalAssists"\
, "goalSaves"\
, "totalPasses"\
, "keyPasses"\
, "passAccuracy"\
, "totalTackles"\
, "blocks"\
, "interceptions"\
, "duelsTotal"\
, "duelsWon"\
, "dribbleAttempts"\
, "dribblesSuccess"\
, "dribblesPast"\
, "foulsDrawn"\
, "foulsCommitted"\
, "cardsYel"\
, "cardsYelRed"\
, "cardsRed"\
, "penaltyWon"\
, "penaltyCommited"\
, "penaltyScored"\
, "penaltyMissed"\
, "penaltySaved"]

In [69]:
#finalPageItems
pagesNum = len(jsonData)
finalPageItems = len(jsonData[pagesNum-1]['response'])

In [70]:
finalArr = []
#Keys to iterate through
playerKeys = ['id', 'firstname', 'lastname', 'age', 'nationality', 'height', 'weight', 'injured']
#Create a dict to optimize code
stat = {
    'team':['id','name'],
    'games':['appearences','lineups','minutes','position','rating','captain'],
    'substitutes':['in','out','bench'],
    'shots':['total','on'],
    'goals':['total','conceded','assists','saves'],
    'passes':['total','key','accuracy'],
    'tackles':['total','blocks','interceptions'],
    'duels':['total','won'],
    'dribbles':['attempts','success','past'],
    'fouls':['drawn','committed'],
    'cards':['yellow','yellowred','red'],
    'penalty':['won','commited','scored','missed','saved']
}

#To change before every extraction
iterable = 0
pagesNum = len(jsonData)
finalPageItems = len(jsonData[pagesNum-1]['response'])
for i in range(pagesNum):
    
    #To adjust for the last page
    if i == pagesNum-1: 
        iterable = finalPageItems
    else:
        iterable = 20
        
    for j in range(iterable):
        to_fill = []
        
        for pkey in playerKeys:
            print(i,j,pkey)
            to_fill.append(jsonData[i]['response'][j]['player'][pkey])
        for key in stat:
            for innerkey in stat[key]:
                to_fill.append(jsonData[i]['response'][j]['statistics'][0][key][innerkey])        
        finalArr.append(to_fill)

0 0 id
0 0 firstname
0 0 lastname
0 0 age
0 0 nationality
0 0 height
0 0 weight
0 0 injured
0 1 id
0 1 firstname
0 1 lastname
0 1 age
0 1 nationality
0 1 height
0 1 weight
0 1 injured
0 2 id
0 2 firstname
0 2 lastname
0 2 age
0 2 nationality
0 2 height
0 2 weight
0 2 injured
0 3 id
0 3 firstname
0 3 lastname
0 3 age
0 3 nationality
0 3 height
0 3 weight
0 3 injured
0 4 id
0 4 firstname
0 4 lastname
0 4 age
0 4 nationality
0 4 height
0 4 weight
0 4 injured
0 5 id
0 5 firstname
0 5 lastname
0 5 age
0 5 nationality
0 5 height
0 5 weight
0 5 injured
0 6 id
0 6 firstname
0 6 lastname
0 6 age
0 6 nationality
0 6 height
0 6 weight
0 6 injured
0 7 id
0 7 firstname
0 7 lastname
0 7 age
0 7 nationality
0 7 height
0 7 weight
0 7 injured
0 8 id
0 8 firstname
0 8 lastname
0 8 age
0 8 nationality
0 8 height
0 8 weight
0 8 injured
0 9 id
0 9 firstname
0 9 lastname
0 9 age
0 9 nationality
0 9 height
0 9 weight
0 9 injured
0 10 id
0 10 firstname
0 10 lastname
0 10 age
0 10 nationality
0 10 height
0 10 

7 14 age
7 14 nationality
7 14 height
7 14 weight
7 14 injured
7 15 id
7 15 firstname
7 15 lastname
7 15 age
7 15 nationality
7 15 height
7 15 weight
7 15 injured
7 16 id
7 16 firstname
7 16 lastname
7 16 age
7 16 nationality
7 16 height
7 16 weight
7 16 injured
7 17 id
7 17 firstname
7 17 lastname
7 17 age
7 17 nationality
7 17 height
7 17 weight
7 17 injured
7 18 id
7 18 firstname
7 18 lastname
7 18 age
7 18 nationality
7 18 height
7 18 weight
7 18 injured
7 19 id
7 19 firstname
7 19 lastname
7 19 age
7 19 nationality
7 19 height
7 19 weight
7 19 injured
8 0 id
8 0 firstname
8 0 lastname
8 0 age
8 0 nationality
8 0 height
8 0 weight
8 0 injured
8 1 id
8 1 firstname
8 1 lastname
8 1 age
8 1 nationality
8 1 height
8 1 weight
8 1 injured
8 2 id
8 2 firstname
8 2 lastname
8 2 age
8 2 nationality
8 2 height
8 2 weight
8 2 injured
8 3 id
8 3 firstname
8 3 lastname
8 3 age
8 3 nationality
8 3 height
8 3 weight
8 3 injured
8 4 id
8 4 firstname
8 4 lastname
8 4 age
8 4 nationality
8 4 height


11 16 injured
11 17 id
11 17 firstname
11 17 lastname
11 17 age
11 17 nationality
11 17 height
11 17 weight
11 17 injured
11 18 id
11 18 firstname
11 18 lastname
11 18 age
11 18 nationality
11 18 height
11 18 weight
11 18 injured
11 19 id
11 19 firstname
11 19 lastname
11 19 age
11 19 nationality
11 19 height
11 19 weight
11 19 injured
12 0 id
12 0 firstname
12 0 lastname
12 0 age
12 0 nationality
12 0 height
12 0 weight
12 0 injured
12 1 id
12 1 firstname
12 1 lastname
12 1 age
12 1 nationality
12 1 height
12 1 weight
12 1 injured
12 2 id
12 2 firstname
12 2 lastname
12 2 age
12 2 nationality
12 2 height
12 2 weight
12 2 injured
12 3 id
12 3 firstname
12 3 lastname
12 3 age
12 3 nationality
12 3 height
12 3 weight
12 3 injured
12 4 id
12 4 firstname
12 4 lastname
12 4 age
12 4 nationality
12 4 height
12 4 weight
12 4 injured
12 5 id
12 5 firstname
12 5 lastname
12 5 age
12 5 nationality
12 5 height
12 5 weight
12 5 injured
12 6 id
12 6 firstname
12 6 lastname
12 6 age
12 6 nationality

16 1 firstname
16 1 lastname
16 1 age
16 1 nationality
16 1 height
16 1 weight
16 1 injured
16 2 id
16 2 firstname
16 2 lastname
16 2 age
16 2 nationality
16 2 height
16 2 weight
16 2 injured
16 3 id
16 3 firstname
16 3 lastname
16 3 age
16 3 nationality
16 3 height
16 3 weight
16 3 injured
16 4 id
16 4 firstname
16 4 lastname
16 4 age
16 4 nationality
16 4 height
16 4 weight
16 4 injured
16 5 id
16 5 firstname
16 5 lastname
16 5 age
16 5 nationality
16 5 height
16 5 weight
16 5 injured
16 6 id
16 6 firstname
16 6 lastname
16 6 age
16 6 nationality
16 6 height
16 6 weight
16 6 injured
16 7 id
16 7 firstname
16 7 lastname
16 7 age
16 7 nationality
16 7 height
16 7 weight
16 7 injured
16 8 id
16 8 firstname
16 8 lastname
16 8 age
16 8 nationality
16 8 height
16 8 weight
16 8 injured
16 9 id
16 9 firstname
16 9 lastname
16 9 age
16 9 nationality
16 9 height
16 9 weight
16 9 injured
16 10 id
16 10 firstname
16 10 lastname
16 10 age
16 10 nationality
16 10 height
16 10 weight
16 10 injured


20 0 nationality
20 0 height
20 0 weight
20 0 injured
20 1 id
20 1 firstname
20 1 lastname
20 1 age
20 1 nationality
20 1 height
20 1 weight
20 1 injured
20 2 id
20 2 firstname
20 2 lastname
20 2 age
20 2 nationality
20 2 height
20 2 weight
20 2 injured
20 3 id
20 3 firstname
20 3 lastname
20 3 age
20 3 nationality
20 3 height
20 3 weight
20 3 injured
20 4 id
20 4 firstname
20 4 lastname
20 4 age
20 4 nationality
20 4 height
20 4 weight
20 4 injured
20 5 id
20 5 firstname
20 5 lastname
20 5 age
20 5 nationality
20 5 height
20 5 weight
20 5 injured
20 6 id
20 6 firstname
20 6 lastname
20 6 age
20 6 nationality
20 6 height
20 6 weight
20 6 injured
20 7 id
20 7 firstname
20 7 lastname
20 7 age
20 7 nationality
20 7 height
20 7 weight
20 7 injured
20 8 id
20 8 firstname
20 8 lastname
20 8 age
20 8 nationality
20 8 height
20 8 weight
20 8 injured
20 9 id
20 9 firstname
20 9 lastname
20 9 age
20 9 nationality
20 9 height
20 9 weight
20 9 injured
20 10 id
20 10 firstname
20 10 lastname
20 10 

27 15 lastname
27 15 age
27 15 nationality
27 15 height
27 15 weight
27 15 injured
27 16 id
27 16 firstname
27 16 lastname
27 16 age
27 16 nationality
27 16 height
27 16 weight
27 16 injured
27 17 id
27 17 firstname
27 17 lastname
27 17 age
27 17 nationality
27 17 height
27 17 weight
27 17 injured
27 18 id
27 18 firstname
27 18 lastname
27 18 age
27 18 nationality
27 18 height
27 18 weight
27 18 injured
27 19 id
27 19 firstname
27 19 lastname
27 19 age
27 19 nationality
27 19 height
27 19 weight
27 19 injured
28 0 id
28 0 firstname
28 0 lastname
28 0 age
28 0 nationality
28 0 height
28 0 weight
28 0 injured
28 1 id
28 1 firstname
28 1 lastname
28 1 age
28 1 nationality
28 1 height
28 1 weight
28 1 injured
28 2 id
28 2 firstname
28 2 lastname
28 2 age
28 2 nationality
28 2 height
28 2 weight
28 2 injured
28 3 id
28 3 firstname
28 3 lastname
28 3 age
28 3 nationality
28 3 height
28 3 weight
28 3 injured
28 4 id
28 4 firstname
28 4 lastname
28 4 age
28 4 nationality
28 4 height
28 4 weigh

In [45]:
len(finalArr)
#len(proto)

703

In [46]:
#TO NOTE: Year in query is season XXXX/XXXX+1
#Example if 2020 is past as param, the data returned is from the season 2020/2021
#Create a 2d array
arr = np.array(finalArr)
arr

array([[17766, 'Luke', 'Coddington', ..., None, None, None],
       [18585, 'Dan', 'Woodards', ..., None, None, None],
       [18593, 'Ricky', 'Shakes', ..., None, None, None],
       ...,
       [19236, 'Muhamed', 'Bešić', ..., 0, 0, None],
       [37154, 'Jordy', 'Clasie', ..., None, None, None],
       [44827, 'Ádám', 'Bogdán', ..., None, None, None]], dtype=object)

In [47]:
#create the df
df = pd.DataFrame(arr, columns = proto)
df

,id,firstname,lastname,age,nationality,height,weight,injured,teamID,teamname,...,foulsDrawn,foulsCommitted,cardsYel,cardsYelRed,cardsRed,penaltyWon,penaltyCommited,penaltyScored,penaltyMissed,penaltySaved
0,17766,Luke,Coddington,26,England,181 cm,83 kg,False,37,Huddersfield,...,None,None,0,0,0,None,None,None,None,None
1,18585,Dan,Woodards,38,England,180 cm,80 kg,False,1830,Boreham Wood,...,None,None,0,0,0,None,None,None,None,None
2,18593,Ricky,Shakes,36,Guyana,178 cm,76 kg,False,1830,Boreham Wood,...,None,None,0,0,0,None,None,None,None,None
3,18774,Wes,Morgan,37,Jamaica,186 cm,101 kg,False,46,Leicester,...,12,31,4,0,0,None,None,0,0,None
4,18879,Andrew,Surman,35,England,178 cm,73 kg,False,35,Bournemouth,...,14,9,2,0,0,None,None,0,0,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
698,30531,Jérémie,Boga,25,Côte d'Ivoire,172 cm,68 kg,False,49,Chelsea,...,0,1,0,0,0,None,None,0,0,None
699,47181,Antonio,Martínez López,25,Spain,180 cm,70 kg,False,48,West Ham,...,None,None,0,0,0,None,None,None,None,None
700,19236,Muhamed,Bešić,30,Bosnia and Herzegovina,180 cm,80 kg,False,45,Everton,...,2,1,0,0,0,None,None,0,0,None
701,37154,Jordy,Clasie,31,Netherlands,169 cm,68 kg,False,41,Southampton,...,None,None,0,0,0,None,None,None,None,None


In [48]:
gb = df.groupby(df['lastname']).count()#=='Kane'].goalAssists
df[df['lastname']=='Morgan']

,id,firstname,lastname,age,nationality,height,weight,injured,teamID,teamname,...,foulsDrawn,foulsCommitted,cardsYel,cardsYelRed,cardsRed,penaltyWon,penaltyCommited,penaltyScored,penaltyMissed,penaltySaved
3,18774,Wes,Morgan,37,Jamaica,186 cm,101 kg,False,46,Leicester,...,12,31,4,0,0,None,None,0,0,None


In [49]:
df['season'] = '2016/2017'

In [50]:
#Export the df to a csv
df.to_csv('data2016&2017.csv')